In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 1. Load Data
df = pd.read_csv('../data/solar_data.csv')
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())

# 2. Data Overview
print("=== DATA OVERVIEW ===")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

# 3. Summary Statistics & Missing Values
print("=== SUMMARY STATISTICS ===")
print(df.describe())

print("\n=== MISSING VALUES ===")
missing_data = df.isna().sum()
missing_percent = (missing_data / len(df)) * 100
missing_report = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_report)

# Identify columns with >5% nulls
high_missing = missing_report[missing_report['Missing Percentage'] > 5]
print(f"\nColumns with >5% missing values: {high_missing.index.tolist()}")

# 4. Outlier Detection using Z-scores
print("=== OUTLIER DETECTION ===")
numeric_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
z_scores = np.abs(stats.zscore(df[numeric_cols].dropna()))
outliers = (z_scores > 3).any(axis=1)
print(f"Number of outlier rows: {outliers.sum()}")

# 5. Data Cleaning
print("=== DATA CLEANING ===")
df_clean = df.copy()

# Handle missing values in key columns
key_columns = ['GHI', 'DNI', 'DHI', 'Tamb', 'RH', 'WS']
for col in key_columns:
    if col in df_clean.columns:
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)

# Remove outliers
df_clean = df_clean[~outliers]

print(f"Original data shape: {df.shape}")
print(f"Cleaned data shape: {df_clean.shape}")

# 6. Time Series Analysis
print("=== TIME SERIES ANALYSIS ===")
df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
df_clean.set_index('Timestamp', inplace=True)

# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
df_clean['GHI'].resample('D').mean().plot(ax=axes[0,0], title='Daily Average GHI')
df_clean['DNI'].resample('D').mean().plot(ax=axes[0,1], title='Daily Average DNI')
df_clean['DHI'].resample('D').mean().plot(ax=axes[1,0], title='Daily Average DHI')
df_clean['Tamb'].resample('D').mean().plot(ax=axes[1,1], title='Daily Average Temperature')
plt.tight_layout()
plt.show()

# 7. Cleaning Impact Analysis
print("=== CLEANING IMPACT ===")
if 'Cleaning' in df_clean.columns:
    cleaning_impact = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()
    print(cleaning_impact)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    cleaning_impact['ModA'].plot(kind='bar', ax=axes[0], title='ModA - Pre/Post Cleaning')
    cleaning_impact['ModB'].plot(kind='bar', ax=axes[1], title='ModB - Pre/Post Cleaning')
    plt.tight_layout()
    plt.show()

# 8. Correlation Analysis
print("=== CORRELATION ANALYSIS ===")
corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS']
corr_matrix = df_clean[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

# 9. Scatter Plots
print("=== RELATIONSHIP ANALYSIS ===")
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
df_clean.plot.scatter(x='WS', y='GHI', ax=axes[0,0], alpha=0.5, title='WS vs GHI')
df_clean.plot.scatter(x='RH', y='Tamb', ax=axes[0,1], alpha=0.5, title='RH vs Temperature')
df_clean.plot.scatter(x='RH', y='GHI', ax=axes[1,0], alpha=0.5, title='RH vs GHI')
df_clean.plot.scatter(x='Tamb', y='GHI', ax=axes[1,1], alpha=0.5, title='Temperature vs GHI')
plt.tight_layout()
plt.show()

# 10. Distribution Analysis
print("=== DISTRIBUTION ANALYSIS ===")
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
df_clean['GHI'].hist(ax=axes[0,0], bins=50)
axes[0,0].set_title('GHI Distribution')
df_clean['WS'].hist(ax=axes[0,1], bins=50)
axes[0,1].set_title('Wind Speed Distribution')
df_clean['Tamb'].hist(ax=axes[1,0], bins=50)
axes[1,0].set_title('Temperature Distribution')
df_clean['RH'].hist(ax=axes[1,1], bins=50)
axes[1,1].set_title('Relative Humidity Distribution')
plt.tight_layout()
plt.show()

# 11. Bubble Chart
print("=== BUBBLE CHART ===")
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_clean['Tamb'], df_clean['GHI'], 
                     c=df_clean['RH'], s=df_clean['BP']/10, 
                     alpha=0.6, cmap='viridis')
plt.colorbar(scatter, label='Relative Humidity (%)')
plt.xlabel('Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.title('GHI vs Temperature (Bubble size = BP, Color = RH)')
plt.show()

# 12. Save Cleaned Data
df_clean.reset_index(inplace=True)
df_clean.to_csv('../data/benin_clean.csv', index=False)
print("Cleaned data saved to ../data/benin_clean.csv")